In [ ]:
# -*- coding: utf-8 -*-
from miasm.analysis.machine import Machine
from miasm.analysis.binary import Container
from miasm.expression.expression import *
from miasm.core.utils import *
from miasm.core.locationdb import LocationDB
from miasm.arch.x86 import regs
from miasm.ir.symbexec import SymbolicExecutionEngine, get_block
from miasm.expression.simplifications import expr_simp

In [ ]:
!unzip -n -P infected zeusvm.zip

In [ ]:
filename = 'zeusvm.bin'

machine = Machine('x86_32')
loc_db = LocationDB()
with open(filename, 'rb') as fstream:
    cont = Container.from_stream(fstream, loc_db)
bs = cont.bin_stream
mdis = machine.dis_engine(bs, loc_db=cont.loc_db)
ir_arch = machine.ir(mdis.loc_db)

In [ ]:
# Error
mnemonic_array_addr = 0x427018

for i in range(69):
    # Get each handler address from the array
    addr = int(hex(upck32(bs.getbytes(mnemonic_array_addr + i*4, 4))), 16)
    print('*'*40, 'Mnemonic', i, ' addr', addr, '*'*40)
    
    # Generate AsmCFG and IRCFG
    asmcfg = mdis.dis_multiblock(addr)
    ircfg = ir_arch.new_ircfg_from_asmcfg(asmcfg)
    irblock = ircfg.get_block(addr)
    
    # Initialize symbolic execution engine
    sb = SymbolicExecutionEngine(ir_arch)
    addr = sb.eval_updt_irblock(irblock)
    if not isinstance(addr, ExprInt):
        print('Unknown destination %s' % addr)

In [ ]:
# Preparing the initial symbols for regs and mems
symbols_init = dict(regs.regs_init)
initial_symbols = symbols_init.items()
ret_addr = ExprId('RET_ADDR', 32)
vm_pc_init = ExprId('VM_PC_init', 32)
infos = {}
infos[expr_simp(ExprMem(regs.ECX_init, 32))] = vm_pc_init
# Push return addr
infos[expr_simp(ExprMem(regs.ESP_init-ExprInt(4, 32), 32))] = ret_addr
infos[regs.ESP] = expr_simp(regs.ESP_init-ExprInt(4, 32))

for i in range(0, 5):
    infos[expr_simp(ExprMem(regs.ECX_init + ExprInt(4*(i+1), 32), 32))] = ExprId('REG%d' % i, 32)

addition_infos = dict(infos)

# imm
expr_imm8 = expr_simp(ExprMem(vm_pc_init + ExprInt(0x1, 32), 8))
addition_infos[expr_imm8] = ExprId('imm8' , 8)

expr_imm16 = expr_simp(ExprMem(vm_pc_init + ExprInt(0x1, 32), 16))
addition_infos[expr_imm16] = ExprId('imm16' , 16)

expr_imm32 = expr_simp(ExprMem(vm_pc_init + ExprInt(0x1, 32), 32))
addition_infos[expr_imm32] = ExprId('imm32' , 32)

# immb
expr_imm8b = expr_simp(ExprMem(vm_pc_init + ExprInt(0x2, 32), 8))
addition_infos[expr_imm8b] = ExprId('imm8b' , 8)

expr_imm16b = expr_simp(ExprMem(vm_pc_init + ExprInt(0x2, 32), 16))
addition_infos[expr_imm16b] = ExprId('imm16b' , 16)

expr_imm32b = expr_simp(ExprMem(vm_pc_init + ExprInt(0x2, 32), 32))
addition_infos[expr_imm32b] = ExprId('imm32b' , 32)

imms = set([expr_imm8, expr_imm16, expr_imm32,
            expr_imm8b, expr_imm16b, expr_imm32b])

imm8 = ExprId('imm8', 8)

base_regx = expr_simp(regs.ECX_init + (imm8.zeroExtend(32) & ExprInt(0xF, 32)) * ExprInt(4, 32) + ExprInt(0xC, 32))
addition_infos[expr_simp(ExprMem(base_regx, 32))] = ExprId('REGX' , 32)
addition_infos[expr_simp(ExprMem(base_regx, 16))] = ExprId('REGX' , 32)[:16]
addition_infos[expr_simp(ExprMem(base_regx, 8))] = ExprId('REGX' , 32)[:8]

base_regy = expr_simp(regs.ECX_init + (imm8[4:8].zeroExtend(32)) * ExprInt(4, 32) + ExprInt(0xC, 32))
addition_infos[expr_simp(ExprMem(base_regy, 32))] = ExprId('REGY' , 32)
addition_infos[expr_simp(ExprMem(base_regy, 16))] = ExprId('REGY' , 16)[:16]
addition_infos[expr_simp(ExprMem(base_regy, 8))] = ExprId('REGY' , 8)[:8]

In [ ]:
def dump_state(sb):
    print('-'*20, 'State', '-'*20)
    out = {}
    for expr, value in sorted(sb.symbols.items()):
        if (expr, value) in initial_symbols:
            continue
        if (expr, value) in addition_infos:
            continue
        if expr in [regs.zf, regs.cf, regs.nf, regs.of, regs.pf, regs.af,
                    ir_arch.IRDst, regs.EIP]:
            continue
        expr_s = expr_simp(expr.replace_expr(addition_infos))
        expr = expr_s
        value = expr_simp(value.replace_expr(addition_infos))
        if expr == value:
            continue
        out[expr] = value

    out = sorted(out.items())
    x86_regs = []
    mem = []
    other = []
    for expr, value in out:
        if expr in regs.all_regs_ids:
            x86_regs.append((expr, value))
        elif isinstance(expr, ExprMem):
            mem.append((expr, value))
        else:
            other.append((expr, value))

    print('Regs:')
    for item in other:
        print('\t%s = %s' % item)
    print('Mem:')
    for item in mem:
        print('\t%s = %s' % item)
    print('x86:')
    for item in x86_regs:
        print('\t%s = %s' % item)
    print('')

In [ ]:
mnemonic_array_addr = 0x427018

for i in range(69):
    # Get each handler address from the array
    addr = int(hex(upck32(bs.getbytes(mnemonic_array_addr + 4*i, 4))), 16)
    print('*'*40, 'Mnemonic', i, ' addr', hex(addr), '*'*40)
    
    # Generate AsmCFG and IRCFG
    asmcfg = mdis.dis_multiblock(addr)
    ircfg = ir_arch.new_ircfg_from_asmcfg(asmcfg)
    irblock = ircfg.get_block(addr)
        
    # Initialize symbolic execution engine
    sb = SymbolicExecutionEngine(ir_arch, symbols_init)
    for k, v in infos.items():
        sb.symbols[k] = v

    symbols = frozenset(sb.symbols.items())
    todo = set([(addr, symbols)])

    count = 20
    while todo and count > 0:
        count -=1
        
        addr, symbols = todo.pop()
        
        irblock = ircfg.get_block(addr)    
        if not irblock:
            print(ValueError('Unknown destination %s' % addr))
            continue

        sb.symbols.symbols_id.clear()
        sb.symbols.symbols_mem.clear()
        for k, v in symbols:
            sb.symbols[k] = v

        print('Block', addr)
        addr = sb.eval_updt_irblock(irblock)

        sb.del_mem_above_stack(ir_arch.sp)

        if addr is ret_addr:
            print('Ret addr reached')
            ret_mn = expr_simp(sb.eval_expr(regs.EAX[:8]))
            if ret_mn != ExprInt(1, 8):
                print('Strange return', ret_mn)
            dump_state(sb)
            continue

        if isinstance(addr, ExprCond):
            todo.add((addr.src1, frozenset(sb.symbols.items())))
            todo.add((addr.src2, frozenset(sb.symbols.items())))
            continue
        if not isinstance(addr, ExprInt) or isinstance(addr, ExprId):
            print('BAD END', addr)
            break

        todo.add((addr, frozenset(sb.symbols.items())))
    if count == 0:
        print('Mnemonic too complex')
